In [12]:
from sage.all_cmdline import *

from sage.all import *

PRIME = 65537
F = GF(PRIME)

# This is to write polynomials in a comfortable way
R = PolynomialRing(F,'x')
X = R.monomial(1)

# Shamir Secret Sharing

The objective of this algorithm is to distribute a secret key among $N$ different parties in such a way that there's a number $t<N$ such that any $t+1$ or more parts can retrieve the secret but $t$ or less cannot.

Formally, we want two functions to exist:
- $S: \mathbb{F}_p \rightarrow \mathbb{F}_p^N$ (called the Share function) that produces $N$ different shares that encode the secret passed as input
- $R: \mathbb{F}_p^s \rightarrow \mathbb{F}_p$ (called the Retrieve function) that, with any subset of $s>t$ shares, can obtain the secret.

This is achieved using the fact that a polynomial of degree $t$ can be unequivocally determined by a set of $t+1$ evaluations, but not less.

Furthermore, there's a method called the Lagrange Interpolation to reconstruct the polynomial from the $t+1$ evaluations $\{(x_1, y_1), (x_2, y_2), \dots , (x_{t+1}, y_{t+1})\}$.

Here we won't prove this fact, but we'll see how to construct the interpolating polynomial.

## Lagrange interpolation

Let's say we have an unknown polynomial $P$ of degree $t$, and we're given a set of evaluations $\{(x_1, y_1), (x_2, y_2), \dots , (x_{t+1}, y_{t+1})\}$ such that for each evaluation, $P(x_i) = y_i$.

The strategy will be to build $t+1$ polynomials $L_i(x)$ such that:
- $L_i(x_i) = 1$
- $L_j(x_i) = 0$ for all $j\neq i$

What for? Consider the following polynomial:

$L(x) = L_1(x)y_1 + L_2(x)y_2 + \dots + L_{t+1}(x)y_{t+1}$ 

If we evaluate it in, say, $L(x_i)$, then $L_i(x_i) = 1$ and $L_i(x_i)y_i = y_i$. All the other $L_j(x_i)$ are 0. Then:

$L(x_i) = y_i$

When we give the formula for the $L_i$ we'll see that the degree of $L(x)$ is also at most $t$, so this means $P$ and $L$ are the same polynomial.

Let's construct the $L_i$ then.

We can start by constructing a polynomial $P_i(x)$ that evaluates to 0 in all the $x_j$ such that $i\neq j$. This is equivalent to say that all those $x_j$ are $P_i$'s roots. So we can write it as:

$P_i(x) = (x - x_1)\dots (x - x_{i-1})(x - x_{i+1})\dots(x - x_{t+1}) = \prod_{j\in\{1\dots t+1\}\land i\neq j} (x - x_j)$

Notice that the term $(x - x_i)$ doesn't appear in the formula.

We're almost done. $P_i(x_i)$ doesn't need to be $1$. It's not 0 though, because, by the way we constructed it, $x_i$ isn't a root. So it's safe to divide the formula by $P_i(x_i)$. Finally, we get:

$L_i(x) = \frac{P_i(x)}{P_i(x_i)} = \frac{\prod_{j\in\{1\dots t+1\}\land i\neq j} (x - x_j)}{\prod_{j\in\{1\dots t+1\}\land i\neq j} (x_i - x_j)}$

In [35]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    @classmethod
    def evaluation(cls, poly, x):
        return cls(x, poly(x))
    
    def __repr__(self):
        return f"({self.x}, {self.y})"
        
class Interpolator:
    def interpolate(self, points):
        # Complete using the function below as a helper
        
    def lagrange_coefficient(self, i, points):
        # Complete
        

original_polynomial = X**3 + 8*X**2 + 5*X + 1
degree = original_polynomial.degree()
print(f"The original polynomial is: {original_polynomial} of degree {degree}")

evaluations = [Point.evaluation(original_polynomial, i+1) for i in range(degree+1)]
print(f"Evaluations are: {evaluations}")

interpolated_polynomial = Interpolator().interpolate(evaluations)
print(f"The interpolated polynomial is: {interpolated_polynomial} of degree {interpolated_polynomial.degree()}")

The original polynomial is: x^3 + 8*x^2 + 5*x + 1 of degree 3
Evaluations are: [(1, 15), (2, 51), (3, 115), (4, 213)]
The interpolated polynomial is: x^3 + 8*x^2 + 5*x + 1 of degree 3


## The protocol

We want to construct a polynomial that can be splitted into some evaluations to distribute as shares, and also that can encode a secret.

We'll encode the secret number $s$ in the coefficient $a_0$ because it's easy to retrieve it once you have the polynomial (it's $P(0)$).

We need $P$ to be of degree $t$ because we want any subset of $t+1$ parties to be able to reconstruct it. So we choose random numbers of $\mathbb{F}_p$ to be $a_1, \dots , a_t$.

The polynomial is then:

$P(x) = $a_t$x^t + \dots + $a_1$x + s$.

The distributed shares are any $N$ evaluations of $P$. We will distribute them as pairs $(x_i, y_i)$.

Notice that, in order to keep the secret, $(0,P(0))$ can't be a share because $P(0)=s$.

The retrieve function is obtained by interpolating the input points and returning $L(0)$.

In [45]:
import random

class ShamirSecretSharing:
    def __init__(self, parties, N):
        self.t = parties - 1
        self.N = N
    
    def share(self, secret):
        # Complete: create the random polynomial, hide the secret and create the shares
    
    def retrieve(self, shares):
        if self.t >= len(shares):
            raise Exception(f"Not enough shares to retrieve the secret ({len(shares)} of {self.t+1}).")
            
        # Complete: recover the polynomial and output the secret

    

original_secret = 100
sss = ShamirSecretSharing(3, 5)

shares = sss.share(original_secret)
print(f"Shares are {shares}")

obtained_secret = sss.retrieve([shares[0], shares[1], shares[2]])
print(f"Obtained secret is {obtained_secret}")

Generated random poly: 4684*x^2 + 34732*x + 100
Shares are [(1, 39516), (2, 22763), (3, 15378), (4, 17361), (5, 28712)]
Obtained secret is 100
